In [218]:
import pickle
import itertools
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pathlib
import csv

In [219]:
pcd = o3d.io.read_point_cloud("GmJMC025_02_ds.ply")
pt_list_np = np.asarray(pcd.points)


In [220]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    dbscan_labels = np.array(pcd.cluster_dbscan(eps=0.1, min_points=100))

max_label = dbscan_labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(dbscan_labels / (max_label if max_label > 0 else 1))
colors[dbscan_labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
#o3d.visualization.draw_geometries([pcd])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 9
point cloud has 9 clusters


In [221]:
pcd.points = o3d.utility.Vector3dVector(pt_list_np[dbscan_labels != -1])
#o3d.visualization.draw_geometries([pcd])

In [222]:
idx_l = []
with open("GmJMC025_02_ds.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        row = list(map(int, row[0].split(" ")[:-1]))
        idx_l.append(row)

In [223]:
labels = np.zeros(len(pt_list_np))

In [224]:
for i, idx in enumerate(idx_l):
    labels[idx] = i+1

In [225]:
labels = labels[dbscan_labels != -1]

In [226]:
for i, label in enumerate(np.unique(labels)):
    labels[labels == label] = i

In [227]:
max_label = labels.max()
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])


In [230]:
o3d.visualization.draw_geometries([pcd])

In [100]:
o3d.io.write_point_cloud("GmJMC025_02_ds_denoise.ply", pcd)

True

In [98]:
with open('point_cloud_label.pickle', 'wb') as f:
    pickle.dump(labels, f)

In [3]:
pcd = o3d.io.read_point_cloud("GmJMC025_02_ds_denoise.ply")
o3d.visualization.draw_geometries([pcd])


[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 要求された変換操作はサポートされていません。 
